In [1]:
    # MTG 1/6
    #POStagとENTtagの間には何らかの関係性が存在？
    #NERにおけるPOStagアプローチはsimpleと何も変わらないのでは＝＞誤差レベルの違いである可能性が高い
    #構文木のほうにいけるとよいのですが...

In [8]:
import torch
device = torch.device("cuda:0")

# data 拡張

## train

In [9]:
with open('data_original/train.txt', 'r', encoding='UTF-8') as f:
    data = f.read()
    data = data.split("\n\n")
    # data = data[1:14987] #不要部分をカット

In [10]:
# storing conll format data in multiple list
document = []
for sentence in data:#14988 sentences
    
    new_sentence_list = []
    sentence_list = sentence.split("\n")
    
    for token in sentence_list:
        token_list = token.split("\t")
        new_sentence_list.append(token_list)
    document.append(new_sentence_list)
    
    # [[['-DOCSTART-', '-X-', '-X-', 'O']],
    #  [['EU', 'NNP', 'B-NP', 'B-ORG'],
    #   ['rejects', 'VBZ', 'B-VP', 'O'],
    #   ['German', 'JJ', 'B-NP', 'B-MISC'],...

In [11]:
document[0]

[['Paul', 'O'], ['International', 'O'], ['airport', 'O'], ['.', 'O']]

### 形態素解析情報を新たに付与

In [12]:
from nltk.parse.stanford import *
import nltk
from nltk.tree import *
import svgling

In [52]:
java_path = "C:/Program Files/Java/jre1.8.0_321/bin/java.exe"
os.environ['JAVAHOME'] = java_path

parser =  'stanford-corenlp/stanford-parser-full-2020-11-17/stanford-parser.jar'
parser_model = 'stanford-corenlp/stanford-corenlp-4.2.0-models-english.jar'

#POSタグの分析用
pos = StanfordParser(path_to_jar=parser, path_to_models_jar = parser_model)
#係り受け関係の分析用
dep_parser = StanfordDependencyParser(path_to_jar=parser, path_to_models_jar = parser_model)

def POSTagAnalysis(text):
    out = pos.raw_parse(text)
    out = list(out)
    tree = out[0]
    return tree

def dependenceAnalysis(text):
    out = dep_parser.raw_parse(text)
    out = list(out)
    parse = out[0]
    return parse

C:\Users\momo1\AppData\Local\Temp\ipykernel_18180\4148535018.py:8: DeprecationWarning: The StanfordParser will be deprecated
Please use nltk.parse.corenlp.CoreNLPParser instead.
  pos = StanfordParser(path_to_jar=parser, path_to_models_jar = parser_model)
C:\Users\momo1\AppData\Local\Temp\ipykernel_18180\4148535018.py:10: DeprecationWarning: The StanfordDependencyParser will be deprecated
Please use nltk.parse.corenlp.CoreNLPDependencyParser instead.
  dep_parser = StanfordDependencyParser(path_to_jar=parser, path_to_models_jar = parser_model)


In [135]:
text = " ".join([token[0] for token in document[1]])
text

"It starred Hicks 's wife , Ellaline Terriss and Edmund Payne ."

In [136]:
tree = POSTagAnalysis(text)
# t = pos.raw_parse(text)

In [138]:
# import inspect
# print(type(tree))
# for x in inspect.getmembers(tree, inspect.ismethod):
#     print(x[0])

In [139]:
# print(tree)
# # 1語ずつのPOSタグ
tree.subtrees()

<generator object Tree.subtrees at 0x000001EEF91DF200>

In [140]:
tree.fromlist

<bound method Tree.fromlist of <class 'nltk.tree.tree.Tree'>>

In [141]:
len(tree[0])

3

In [142]:
# # 木構造での出力
print(tree.pretty_print())

                               ROOT                                              
                                |                                                 
                                S                                                
  ______________________________|______________________________________________   
 |                              VP                                             | 
 |      ________________________|______                                        |  
 |     |                               NP                                      | 
 |     |                _______________|_____________________________          |  
 |     |               NP       |             |           |          |         | 
 |     |            ___|___     |             |           |          |         |  
 NP    |           NP      |    |             NP          |          NP        | 
 |     |       ____|___    |    |       ______|_____      |     _____|____     |  
PRP   VBD 

In [143]:
print(tree)
print(type(tree))

(ROOT
  (S
    (NP (PRP It))
    (VP
      (VBD starred)
      (NP
        (NP (NP (NNP Hicks) (POS 's)) (NN wife))
        (, ,)
        (NP (NNP Ellaline) (NNP Terriss))
        (CC and)
        (NP (NNP Edmund) (NNP Payne))))
    (. .)))
<class 'nltk.tree.tree.Tree'>


In [144]:
tree.height()

8

In [147]:
tree.treepositions()

[(),
 (0,),
 (0, 0),
 (0, 0, 0),
 (0, 0, 0, 0),
 (0, 1),
 (0, 1, 0),
 (0, 1, 0, 0),
 (0, 1, 1),
 (0, 1, 1, 0),
 (0, 1, 1, 0, 0),
 (0, 1, 1, 0, 0, 0),
 (0, 1, 1, 0, 0, 0, 0),
 (0, 1, 1, 0, 0, 1),
 (0, 1, 1, 0, 0, 1, 0),
 (0, 1, 1, 0, 1),
 (0, 1, 1, 0, 1, 0),
 (0, 1, 1, 1),
 (0, 1, 1, 1, 0),
 (0, 1, 1, 2),
 (0, 1, 1, 2, 0),
 (0, 1, 1, 2, 0, 0),
 (0, 1, 1, 2, 1),
 (0, 1, 1, 2, 1, 0),
 (0, 1, 1, 3),
 (0, 1, 1, 3, 0),
 (0, 1, 1, 4),
 (0, 1, 1, 4, 0),
 (0, 1, 1, 4, 0, 0),
 (0, 1, 1, 4, 1),
 (0, 1, 1, 4, 1, 0),
 (0, 2),
 (0, 2, 0)]

In [153]:
tree[0][1][1][4].pprint()

(NP (NNP Edmund) (NNP Payne))


In [145]:
l = tree.productions()

In [146]:
l[1]

S -> NP VP .

In [115]:
tree.set_label("NP")

In [110]:
tree.pformat_latex_qtree()

'\\Tree [.ROOT\n        [.FRAG\n          [.NP [.NNP Paul ] [.NNP International ] ]\n          [.NP [.NN airport ] ]\n          [.. . ] ] ]'

In [121]:
tree.fromlist()

In [116]:
tree.leaves()

['Paul', 'International', 'airport', '.']

In [119]:
tree.chomsky_normal_form()

In [122]:
tree.leaf_treeposition()

TypeError: leaf_treeposition() missing 1 required positional argument: 'index'

In [18]:
# POSタグの情報
# print(tree)
# # 1語ずつのPOSタグ
tree.pos()
# # 木構造での出力
print(tree.pretty_print())

              ROOT                
               |                   
              FRAG                
       ________|________________   
      NP                  NP    | 
  ____|________           |     |  
NNP           NNP         NN    . 
 |             |          |     |  
Paul     International airport  . 

None


In [106]:
# make new label
new_document = []
for sentence in document:
    new_sentence_list = []
    for token_list in sentence:
        try:
            new_token_list = [token_list[0],token_list[1]+"_"+token_list[3],token_list[3]]
            new_sentence_list.append(new_token_list)
        except:
            pass
    new_document.append(new_sentence_list)
    
# [[['EU', 'NNP_B-ORG', 'B-ORG'],
#   ['rejects', 'VBZ_O', 'O'],
#   ['German', 'JJ_B-MISC', 'B-MISC'],
#   ['call', 'NN_O', 'O'],...

In [107]:
    # ここで一つの疑問
    # 交換の割合はどのようにすべきだろうか？
    # (1)simple augmentを参照し確認
    # (2)とりあえず10% 30% 50% 70% 100%と順に試してみる => 先に作成してみましょ

In [108]:
# making dictonary object with label as key and token as value
label_token_dic = {}
for sentence in new_document:
    for token_list in sentence:
        label = token_list[1]
        token = token_list[0]
        if label not in label_token_dic:
            label_token_dic[label] = set()
        label_token_dic[label].add(token)
        
    #  {'NNP_B-ORG': {'Detroit',
    #   'FK',
    #   'ATRIA',
    #   'DBRS',...
    
    # label_token_dic.keys() でkey一覧取得可

In [109]:
    # 交換可能パタン
    # B，Iのみ　=> 固有表現に限定したsimple data augmantation
    # oも含む 　=> アノテーションラベル全体に範囲を広げたsimple data augmantation

In [110]:
    # ひとまずB，Iのみから作成

In [111]:
# making BI label list
BI_label_list = [label for label in list(label_token_dic.keys()) if "B-" in label or "I-" in label]
    # ['NNP_B-ORG',
    #  'JJ_B-MISC',
    #  'NNP_B-PER',
    #  'NNP_I-PER',

In [113]:
def Sent2str(sentence_list):
    return " ".join([token_list[0] for token_list in sentence_list])

def random_choice_token(token_set):
    n = random.randint(0,len(token_set)-1)
    token_list=list(token_set)
    return token_list[n]

In [114]:
import random
import csv


def SimpleDA_BI(n,filename):
    data_f_path = "data_augment/SimpleDA_BI/"+str(n)+"_"+filename+".txt"
    memo_f_path = "data_augment/SimpleDA_BI/"+str(n)+"_"+filename+"_memo.txt"
    with open(data_f_path,"w") as data_f, open(memo_f_path,"w") as memo_f:
        for now in range(n):
            if now%10 == 0:
                print('\r%d / %d' %(now, n), end='')

            x = random.randint(0,len(new_document)-1)
            origin_sentence = new_document[x]
            aug_sentence = []
            cnt = 0

            for token_list in origin_sentence:
                token = token_list[0]
                label = token_list[1]
                origin_label = token_list[2]

                if label in BI_label_list:
                    if len(label_token_dic[label])!=1:
                        token=random_choice_token(label_token_dic[label]-{token})
                        cnt = cnt + 1
                    

                aug_sentence.append([token,origin_label])
                data_f.write(" ".join([token,origin_label])+"\n")
            data_f.write("\n")

            memo_f.write(str(x)+"\t"+Sent2str(new_document[x])+"\n")
            memo_f.write(" "*len(str(x))+"\t"+Sent2str(aug_sentence)+"\n")
            memo_f.write("\n")
        
        memo_f.write(str(cnt)+"sentences augmented!!!! \n")

In [115]:
# #train
n = 15000

In [116]:
SimpleDA_BI(n*1,"train")
SimpleDA_BI(n*4,"train")
SimpleDA_BI(n*9,"train")
SimpleDA_BI(n*49,"train")
SimpleDA_BI(n*99,"train")

1484990 / 1485000

## valid 

In [117]:
with open('data_original/valid.txt', 'r', encoding='UTF-8') as f:
    data = f.read()
    data = data.split("\n\n")
    data = data[1:3466]

In [118]:
# 以下同様
document = []
for sentence in data:#14988 sentences
    
    new_sentence_list = []
    sentence_list = sentence.split("\n")
    
    for token in sentence_list:
        token_list = token.split(" ")
        new_sentence_list.append(token_list)
    document.append(new_sentence_list)

new_document = []
for sentence in document:
    new_sentence_list = []
    for token_list in sentence:
        try:
            new_token_list = [token_list[0],token_list[1]+"_"+token_list[3],token_list[3]]
            new_sentence_list.append(new_token_list)
        except:
            pass
    new_document.append(new_sentence_list)

label_token_dic = {}
for sentence in new_document:
    for token_list in sentence:
        label = token_list[1]
        token = token_list[0]
        if label not in label_token_dic:
            label_token_dic[label] = set()
        label_token_dic[label].add(token)

        BI_label_list = [label for label in list(label_token_dic.keys()) if "B-" in label or "I-" in label]

BI_label_list = [label for label in list(label_token_dic.keys()) if "B-" in label or "I-" in label]

In [119]:
#valid
n = 3500

In [120]:
SimpleDA_BI(n*1,"valid")
SimpleDA_BI(n*4,"valid")
SimpleDA_BI(n*9,"valid")
SimpleDA_BI(n*49,"valid")
SimpleDA_BI(n*99,"valid")

346490 / 346500